Αρχικοπιήση και Imports που ειναι απαραιτητα

In [1]:
import json
import os
import sys
import numpy as np
import matplotlib.pyplot as plt


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  #Auto den xreiazetai an exoume polla vms

In [3]:
os.environ.pop('TF_CONFIG', None)

In [4]:
if '.' not in sys.path:
  sys.path.insert(0, '.')

In [8]:
pip install tf-nightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model

Ειαγωγή dataset (Την γράφουμε τον κώδικα της  εισαγωγής  σε ένα python αρχείο το οποίο θα χρησιμοποιηθεί από όλους τους workers)

In [7]:
%%writefile cifar_setup.py

import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model

def cifar_dataset(batch_size):
  # Load in the data
  cifar10 = tf.keras.datasets.cifar10
 
  # Distribute it to train and test set
  (x_train, y_train), (x_test, y_test) = cifar10.load_data()

  # Reduce pixel values
  x_train, x_test = x_train / 255.0, x_test / 255.0
 
  # flatten the label values
  y_train, y_test = y_train.flatten(), y_test.flatten()
  train_dataset = tf.data.Dataset.from_tensor_slices(
      (x_train, y_train)).shuffle(60000).repeat().batch(batch_size)

 
  return train_dataset ,x_train , y_train ,x_test , y_test

def build_and_compile_cnn_model(K, M):

      
  
  
  
  # Build the model using the functional API
  # input layer
  i = Input(shape=M)
  x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
  x = BatchNormalization()(x)
  x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  
  x = Flatten()(x)
  x = Dropout(0.2)(x)
  
  # Hidden layer
  x = Dense(1024, activation='relu')(x)
  x = Dropout(0.2)(x)
  
  # last hidden layer i.e.. output layer
  x = Dense(K, activation='softmax')(x)
  
  model = Model(i, x)
  
  # model description
  model.summary()

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model


Writing cifar_setup.py


Μόνος ένας worker

In [8]:
import cifar_setup
batch_size = 32
single_worker_dataset ,x_train, y_train , x_test ,y_test = cifar_setup.cifar_dataset(batch_size)
K = len(set(y_train))
M=x_train[0].shape
single_worker_model =cifar_setup.build_and_compile_cnn_model(K,M)
tic = perf_counter()
single_worker_model.fit(single_worker_dataset, epochs=3, steps_per_epoch=70)
training_time = perf_counter() - tic
print(f'Total training time: {training_time} secs')

170498071/170498071 [==============================] - 3s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                            

SyntaxError: ignored

In [9]:

data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
  width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
 
train_generator = data_generator.flow(x_train, y_train, batch_size)
steps_per_epoch = x_train.shape[0] // batch_size
tic = perf_counter()

r = single_worker_model.fit(train_generator, validation_data=(x_test, y_test),
              steps_per_epoch=steps_per_epoch, epochs=50)


training_time = perf_counter() - tic
print(f'Total training time: {training_time} secs')

Epoch 1/50
  25/1562 [..............................] - ETA: 8:00 - loss: 1.6131 - accuracy: 0.4412

KeyboardInterrupt: ignored

2 workers

In [ ]:
tf_config = {
    'cluster': {
        'worker': ['localhost:12345', 'localhost:23456']
    },
    'task': {'type': 'worker', 'index': 0}
}

gia to deutero

tf_config['task']['index']=1

to trito

tf_config['task']['index']=2

ktl

In [ ]:
json.dumps(tf_config)

'{"cluster": {"worker": ["localhost:12345", "localhost:23456"]}, "task": {"type": "worker", "index": 0}}'

Ekapideysh tou modelou

In [ ]:
strategy = tf.distribute.MultiWorkerMirroredStrategy()


In [ ]:
with strategy.scope():
  # Model building/compiling need to be within `strategy.scope()`.
  multi_worker_model = cifar_setup.build_and_compile_cnn_model()

In [ ]:
%%writefile main.py

import os
import json

import tensorflow as tf
import cifar_setup

per_worker_batch_size = 64
tf_config = json.loads(os.environ['TF_CONFIG'])
num_workers = len(tf_config['cluster']['worker'])

strategy = tf.distribute.MultiWorkerMirroredStrategy()

global_batch_size = per_worker_batch_size * num_workers
multi_worker_dataset = cifar_setup.mnist_dataset(global_batch_size)

with strategy.scope():
  # Model building/compiling need to be within `strategy.scope()`.
  multi_worker_model = cifar_setup.build_and_compile_cnn_model()


multi_worker_model.fit(multi_worker_dataset, epochs=3, steps_per_epoch=70)

Writing main.py


In [ ]:
%%bash
ls *.py

main.py
mnist_setup.py


In [ ]:
os.environ['TF_CONFIG'] = json.dumps(tf_config)

In [ ]:
# first kill any previous runs
%killbgscripts

All background processes were killed.


In [ ]:
%%bash --bg
python main.py &> job_0.log

In [ ]:
import time
time.sleep(10)

In [ ]:
%%bash
cat job_0.log

2022-09-04 15:03:39.917085: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-09-04 15:03:39.917227: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-09-04 15:03:39.917243: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2022-09-04 15:03:42.023861: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to c

ta apo kato gia ton worker number 2

In [ ]:
tf_config['task']['index'] = 1
os.environ['TF_CONFIG'] = json.dumps(tf_config)

In [ ]:
bash
python main.py

meta kanoume experiment me sharding dld pos na xorisontai ta dta se kommatia

des https://www.tensorflow.org/tutorials/distribute/input#sharding

gia na kaneis xrisi checkpoints h na kaneis save to modelo  kai ta bari xrisimopoies https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/distribute/multi_worker_with_keras.ipynb#scrollTo=XVk4ftYx6JAO